In [1]:
import numpy as np

from qoptcraft.state import Fock, PureState
from qoptcraft.optical_elements import beam_splitter, phase_shifter
from qoptcraft.evolution import photon_unitary

## Photonic circuit and evolution

Create a scattering matrix for 3 modes

In [2]:
modes = 4

scattering = np.eye(modes, dtype=np.complex128)
scattering @= beam_splitter(angle=0.3, shift=0.2, dim=modes, mode_1=0, mode_2=1)
scattering @= beam_splitter(angle=0.5, shift=0, dim=modes, mode_1=0, mode_2=2)
scattering @= phase_shifter(shift=0.5, dim=modes, mode=3)
print(f"{scattering = }")

scattering = array([[ 0.82167472+0.16656171j, -0.29552022+0.j        ,
        -0.44888296-0.09099308j,  0.        +0.j        ],
       [ 0.25417379+0.05152358j,  0.95533651+0.j        ,
        -0.13885578-0.02814746j,  0.        +0.j        ],
       [ 0.47942555+0.j        ,  0.        +0.j        ,
         0.87758255+0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.87758255+0.47942555j]])


Suppose we now want to evolve a Bell state with this scattering matrix

In [3]:
bell_state = (Fock(1, 0, 1, 0) + Fock(0, 1, 0, 1)) / np.sqrt(2)

We first obtain the quantum unitary associated with S for 2 photons:

In [4]:
unitary = photon_unitary(scattering, photons=bell_state.photons)

And then we evolve the state with this unitary:

In [5]:
evolved_bell_state = bell_state.evolution(unitary)
evolved_bell_state

-0.35-0.15j * (2, 0, 0, 0) + 
-0.15-0.07j * (1, 1, 0, 0) + 
0.36+0.07j * (1, 0, 1, 0) + 
-0.18-0.10j * (1, 0, 0, 1) + 
-0.03-0.01j * (0, 2, 0, 0) + 
0.11+0.02j * (0, 1, 1, 0) + 
0.59+0.32j * (0, 1, 0, 1) + 
0.42+0.00j * (0, 0, 2, 0)

## Optimization

Optimization routine to obtain a W state

In [6]:
modes = 9
photons = 6
initial_state = Fock(1, 1, 1, 0, 0, 0, 1, 1, 1)
target_state = (
    Fock(1, 0, 1, 0, 0, 1, 1, 1, 1)
    + Fock(1, 0, 0, 1, 1, 0, 1, 1, 1)
    + Fock(0, 1, 1, 0, 1, 0, 1, 1, 1)
)

We calculate the fidelity of the evolved state with respect to the original one:

In [7]:
from qoptcraft.state import fidelity

fidelity(initial_state, target_state)

np.int64(0)

To constraint the optimization to the linear optical hamiltonians

In [8]:
from numpy.typing import NDArray
import scipy as sp

from qoptcraft.basis import unitary_algebra_basis
from qoptcraft.evolution import fock_evolution

scattering_basis = unitary_algebra_basis(modes)


def parametrize_scattering(params: NDArray):
    dim = int(np.sqrt(params.size))
    hamiltonian = np.zeros((dim, dim), dtype=np.complex128)
    for param, matrix in zip(params, scattering_basis, strict=True):
        hamiltonian += param * matrix
    return sp.linalg.expm(hamiltonian)


def evolve_state(params: NDArray):
    scattering = parametrize_scattering(params)
    return fock_evolution(scattering, list(initial_state))


def cost(params):
    fidelity = np.abs(np.dot(initial_state.state_in_basis().conj(), evolve_state(params))) ** 2
    return 1 - fidelity

In [9]:
evolve_state(np.random.randn(9 * 9))

array([-0.001289  +0.00137715j, -0.00238882+0.00548641j,
        0.0049059 +0.00153762j, ..., -0.00212313+0.01886296j,
        0.0102331 -0.00250372j,  0.00371566-0.00365872j])

In [10]:
cost(np.random.randn(9 * 9))

np.float64(0.9996688277280286)

In [11]:
sp.optimize.minimize(cost, np.random.randn(9 * 9), options={"disp": True})

KeyboardInterrupt: 